In [1]:
import pandas as pd
import numpy as np

# Reading in and splitting the data
bank_marketing=pd.read_csv('bank_marketing.csv')
client=bank_marketing.loc[:,'client_id':'loan']
campaign=bank_marketing[['client_id' , 'campaign' , 'month' , 'day' , 'duration' , 'pdays' , 'previous' , 'poutcome' , 'y']]
economics=bank_marketing[['client_id' , 'emp_var_rate' , 'cons_price_idx' , 'euribor3m' , 'nr_employed']]

In [2]:
# Renaming columns
client=client.rename(columns={'client_id' : 'id'})
campaign=campaign.rename(columns={'duration' : 'contact_duration' , 'previous' : 'previous_campaign_contacts' , 'y': 'campaign_outcome' ,'poutcome' : 'previous_outcome' , 'campaign' : 'number_contacts'})
economics=economics.rename(columns={'euribor3m' : 'euribor_three_months' , 'nr_employed' : 'economics'})

In [3]:
# Cleaning columns
client['education']=client['education'].str.replace('.' , '_')
client['education']=client['education'].replace('unknown' , np.NaN)
client['job']=client['job'].str.replace('.' , '')

campaign['campaign_outcome']=campaign['campaign_outcome'].replace({'no' : 0 , 'yes' : 1})
campaign['previous_outcome']=campaign['previous_outcome'].replace('nonexistent' , np.NaN)

# Creating new columns
campaign['campaign_id']='1'
campaign['month']=campaign['month'].str.capitalize()
campaign['day']=campaign['day'].astype(str)
campaign['year']='2022'
campaign['last_contact_date']=campaign['year'] + '-' + campaign['month'] + '-' + campaign['day']
campaign['last_contact_date']=pd.to_datetime(campaign['last_contact_date'] , format='%Y-%b-%d')

# Deleting unnecessary columns
campaign.drop(columns=['month' , 'day' , 'year'] , inplace=True)



C:\Users\jehsa\AppData\Local\Temp/ipykernel_29084/3181151674.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  client['education']=client['education'].str.replace('.' , '_')
C:\Users\jehsa\AppData\Local\Temp/ipykernel_29084/3181151674.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  client['job']=client['job'].str.replace('.' , '')


In [4]:
# Saving the data
client.to_csv('client.csv' , index=False)
campaign.to_csv('campaign.csv' , index=False)
economics.to_csv('economics.csv' , index=False)

In [6]:
# Designing the database

client_table=""" 
CREATE TABLE client (
    id SERIAL PRIMARY KEY, 
    age INTEGER,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN
);

\copy client FROM 'client.csv' DELIMITER ',' CSV HEADER;

"""

campaign_table=""" 
CREATE TABLE campaign (
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL REFERENCES client(id),
    number_contacts INTEGER,
    contact_duration INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date date
);

\copy campaign FROM 'campaign.csv' DELIMITER ',' CSV HEADER;

"""

economics_table="""
CREATE TABLE economics (
    client_id SERIAL REFERENCES client(id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_empolyed FLOAT
    
);

\copy economics FROM 'economics.csv' DELIMITER ',' CSV HEADER;

"""